# Get alert history for all devices

In [ ]:
from mxnet import nd
from mxnet.gluon.data import DataLoader
from mxnet.gluon.data.dataset import ArrayDataset
from multiprocessing import cpu_count
import pandas as pd
import numpy as np

def load_data_alerts_from_csv(batch_size):
    num_workers = 0
    features = ['f1', 'f2', 'f3', 'f4']
    label = ['a']

    train_features = pd.read_csv('../data/alert/train.csv', usecols=features).values
    train_labels = pd.read_csv('../data/alert/train.csv', usecols=label).values

    test_features  = pd.read_csv('../data/alert/test.csv', usecols=features).values
    test_labels  = pd.read_csv('../data/alert/test.csv', usecols=label).values

    train_iter = DataLoader(
        ArrayDataset(train_features.astype('float32'), train_labels),
        shuffle=True,
        batch_size = batch_size,
        num_workers= num_workers
    )

    test_iter = DataLoader(
        ArrayDataset(test_features.astype('float32'), test_labels),
        batch_size = batch_size,
        num_workers= num_workers
    )
    
    return train_iter, test_iter

In [ ]:
from mxnet import autograd, nd

num_inputs  = 4
num_outputs = 3
batch_size = 200

train_iter, test_iter = load_data_alerts_from_csv(batch_size)

W = nd.random.normal(scale=0.01, shape=(num_inputs, num_outputs))
b = nd.zeros(num_outputs)
params = [W, b]

for param in params:
    param.attach_grad()

In [ ]:
def softmax(X):
    X_exp = nd.exp(X)
    partition = X_exp.sum(axis=1, keepdims=True)
    return X_exp / partition

In [ ]:
def net(X):
    return softmax(nd.dot(X, W) + b)

In [ ]:
def cross_entropy(y_hat, y):
    return - nd.pick(y_hat, y).log()

In [ ]:
def accuracy(y_hat, y):
    return (y_hat.argmax(axis=1) == y.astype('float32')).mean().asscalar()

In [ ]:
def evaluate_accuracy(data_iter, net):
    acc = 0.
    for X, y in data_iter:
        y_hat = net(X)
        acc += accuracy(y_hat, y)
    return acc / len(data_iter)

In [ ]:
def SGD(params, learning_rate):
    for param in params:
        param[:] = param - learning_rate * param.grad

In [ ]:
import pickle

from mxnet import autograd

modelFileName = 'alert_params'

num_epochs, learning_rate = 5000, 0.001

for epoch in range(num_epochs):
    train_loss = 0.
    train_acc = 0.
    for X, y in train_iter:
        with autograd.record():
            y_hat = net(X)
            loss = cross_entropy(y_hat, y)
        loss.backward()
        
        SGD(params, learning_rate / batch_size)
        
        train_loss += loss.mean().asscalar()
        train_acc += accuracy(y_hat, y)
        
    test_acc = evaluate_accuracy(test_iter, net)

    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch + 1, train_loss/len(train_iter), train_acc/len(train_iter), test_acc))
    
    with open(modelFileName, 'wb') as output:
        pickle.dump(params, output)


In [ ]:
params

In [ ]:
import pickle

modelFileName = 'alert_params'

with open(modelFileName, 'wb') as output:
    pickle.dump(params, output)